# Score-Based Generative Modeling in 2D

## Tutorial: Learning Score Functions with Denoising Score Matching

This notebook demonstrates **score-based generative modeling** using Julia, a powerful approach for learning probability distributions and generating samples. We'll work with a 2D example to visualize the concepts clearly.

### What You'll Learn
1. What is a **score function** and why it's useful
2. The **denoising score matching** objective
3. How to train a neural network to approximate scores
4. How to generate samples using **Langevin dynamics**

---


## Dependencies

This tutorial requires the following Julia packages:

| Package | Purpose |
|---------|----------|
| `GLMakie` | Interactive plotting and visualization |
| `LinearAlgebra` | Matrix operations and linear algebra |
| `ProgressBars` | Training progress visualization |
| `Random` | Random number generation |
| `Statistics` | Statistical functions (mean, cov, etc.) |
| `HDF5` | Reading data from HDF5 files |
| `Enzyme` | Automatic differentiation for gradients |

Make sure to activate the project environment before running:


In [1]:
using GLMakie, LinearAlgebra, ProgressBars, Random, Statistics, HDF5

Random.seed!(1234)  # For reproducibility

# Include custom neural network implementations
include("simple_networks.jl")


update! (generic function with 3 methods)

---

## Part 1: Understanding Score Functions

### What is a Score Function?

For a probability distribution $p(\mathbf{x})$, the **score function** is defined as:

$$\mathbf{s}(\mathbf{x}) = \nabla_{\mathbf{x}} \log p(\mathbf{x})$$

The score tells us the direction of steepest increase in log-probability. In other words, it points toward regions of higher probability density.

### Why Use Scores?

1. **Avoids normalization**: Unlike $p(\mathbf{x})$, the score doesn't require computing the partition function $Z = \int p(\mathbf{x}) d\mathbf{x}$
2. **Enables sampling**: We can generate samples using Langevin dynamics
3. **Foundation for diffusion models**: Modern generative AI (DALL-E, Stable Diffusion) builds on these ideas

### Our Target Distribution

We'll work with a 2D distribution defined by the potential energy function:

$$V(\mathbf{x}) = \frac{(x_1^2 - 1)^2}{4} + \frac{(x_2^2 - 1)^2}{4} + \frac{x_1 x_2}{3}$$

The probability distribution is:
$$p(\mathbf{x}) \propto \exp(-V(\mathbf{x}))$$

This creates a **bimodal distribution** with two peaks (modes) due to the $(x^2 - 1)^2$ terms.


In [2]:
# ─────────────────────────────────────────────────────────────────────────────
# Score Functions (2D)
# ─────────────────────────────────────────────────────────────────────────────

"""
Exact score function from the potential V(x) = (x₁²-1)²/4 + (x₂²-1)²/4 + x₁x₂/3

For p(x) ∝ exp(-V(x)), the score is:
    s(x) = ∇log p(x) = -∇V(x)

Computing ∇V:
    ∂V/∂x₁ = x₁³ - x₁ + x₂/3
    ∂V/∂x₂ = x₂³ - x₂ + x₁/3

Therefore:
    s(x) = [x₁ - x₁³ - x₂/3, x₂ - x₂³ - x₁/3]
"""
function exact_score(x)
    x1, x2 = x[1], x[2]
    return [x1 - x1^3 - x2/3, x2 - x2^3 - x1/3]
end


exact_score

### Gaussian Mixture Score (Empirical)

When we only have samples from the distribution (no analytical form), we can estimate the score using a **kernel density estimate** with Gaussian kernels:

$$\hat{p}(\mathbf{x}) = \frac{1}{M} \sum_{i=1}^{M} \mathcal{N}(\mathbf{x} | \mathbf{x}_i, \sigma^2 I)$$

The score of this mixture is:

$$\mathbf{s}_{\text{GM}}(\mathbf{x}) = \frac{\sum_{i=1}^{M} w_i (\mathbf{x}_i - \mathbf{x})}{\sigma^2}$$

where $w_i \propto \exp\left(-\frac{\|\mathbf{x} - \mathbf{x}_i\|^2}{2\sigma^2}\right)$


In [3]:
"""
Gaussian mixture score estimator.

This computes the score of a kernel density estimate formed by placing
Gaussian kernels at each data point.

Arguments:
    x     : 2-element vector (query point)
    data  : 2×M matrix (data samples)
    sigma : kernel bandwidth
"""
function gaussian_mixture_score(x, data, sigma)
    m = size(data, 2)
    score_value = zeros(2)
    denominator = 0.0
    
    for i in 1:m
        Δ = data[:, i] .- x
        U = exp(-(0.5 / sigma^2) * dot(Δ, Δ))
        score_value .+= U .* Δ
        denominator += U
    end
    
    return score_value ./ (denominator * sigma^2)
end


gaussian_mixture_score

---

## Part 2: Denoising Score Matching

### The Challenge

We want to train a neural network $\mathbf{s}_\theta(\mathbf{x})$ to approximate the true score $\nabla_\mathbf{x} \log p(\mathbf{x})$.

**Naive approach (doesn't work):**
$$\mathcal{L}_{\text{naive}} = \mathbb{E}_{p(\mathbf{x})} \left[ \| \mathbf{s}_\theta(\mathbf{x}) - \nabla_\mathbf{x} \log p(\mathbf{x}) \|^2 \right]$$

This requires knowing $\nabla_\mathbf{x} \log p(\mathbf{x})$ — but that's exactly what we're trying to learn!

### The Denoising Score Matching Trick

**Key insight**: Instead of matching the score of the data distribution, we match the score of a **noise-perturbed** distribution!

Given a data point $\mathbf{x}$, we create a noisy version:
$$\tilde{\mathbf{x}} = \mathbf{x} + \sigma \mathbf{z}, \quad \mathbf{z} \sim \mathcal{N}(0, I)$$

The score of the conditional distribution $p(\tilde{\mathbf{x}} | \mathbf{x})$ is:
$$\nabla_{\tilde{\mathbf{x}}} \log p(\tilde{\mathbf{x}} | \mathbf{x}) = -\frac{\tilde{\mathbf{x}} - \mathbf{x}}{\sigma^2} = -\frac{\mathbf{z}}{\sigma}$$

### Denoising Score Matching Loss

The **denoising score matching** objective is:

$$\boxed{\mathcal{L}_{\text{DSM}} = \mathbb{E}_{\mathbf{x} \sim p(\mathbf{x})} \mathbb{E}_{\mathbf{z} \sim \mathcal{N}(0, I)} \left[ \left\| \mathbf{s}_\theta(\mathbf{x} + \sigma\mathbf{z}) + \frac{\mathbf{z}}{\sigma} \right\|^2 \right]}$$

**Why this works:**
- We know $\mathbf{z}$ (we sampled it!)
- The network learns to predict $-\mathbf{z}/\sigma$, which is the score of the noisy distribution
- As $\sigma \to 0$, this converges to the true data score


In [4]:
# ─────────────────────────────────────────────────────────────────────────────
# Loss Functions (2D)
# ─────────────────────────────────────────────────────────────────────────────

"""
Gaussian mixture loss function.

Trains the network to match the Gaussian mixture score estimate.
This is more expensive but provides a ground truth to compare against.
"""
function gaussian_mixture_loss_function(model, data, σ, zs)
    batchsize = size(data, 2)
    lossval = [0.0]
    
    for i in 1:batchsize
        x = data[:, i] .+ σ .* zs[:, i]
        ŷ = model(x)
        y = gaussian_mixture_score(x, data, σ)
        lossval[1] += sum((y .- ŷ).^2) / batchsize
    end
    
    return lossval[1]
end

"""
Denoising Score Matching loss function.

This is the KEY loss function:
    L = E[(s_θ(x̃) + z/σ)²]
    
where x̃ = x + σz and z ~ N(0, I)

The network learns to predict -z/σ, which equals the score of the
noise-perturbed distribution.
"""
function denoising_loss_function(model, data, sigma, noises)
    batchsize = size(data, 2)
    lossval = [0.0]
    
    for i in 1:batchsize
        x = data[:, i]           # Original data point
        z = noises[:, i]         # Random noise
        x̃ = x .+ sigma .* z     # Perturbed point
        ŷ = model(x̃)            # Network prediction
        
        # Loss: (prediction + z/σ)²
        # Network should predict -z/σ
        lossval[1] += sum((ŷ .+ z ./ sigma).^2) / batchsize
    end
    
    return lossval[1]
end

# Use denoising score matching for training
loss_function = denoising_loss_function


denoising_loss_function (generic function with 1 method)

---

## Part 3: Loading and Visualizing the Data

We load samples from our target distribution, generated using Langevin dynamics on the potential $V(\mathbf{x})$.


In [5]:
# ─────────────────────────────────────────────────────────────────────────────
# Load Data and Setup
# ─────────────────────────────────────────────────────────────────────────────

# Load 2D data from HDF5 file
data = h5read("potential_data_2D.hdf5", "data")  # 2×M matrix
M = size(data, 2)
println("Loaded $(M) samples of 2D data")

# Noise level for score matching
sigma = σ = 0.05


Loaded 10000 samples of 2D data


0.05

### Figure 1: Data Distribution

This figure shows the **marginal distributions** of our 2D data along each axis.

**What to observe:**
- **Bimodal structure**: Each dimension shows two peaks around $x \approx \pm 1$
- This comes from the $(x^2 - 1)^2$ terms in the potential, which have minima at $x = \pm 1$
- The coupling term $x_1 x_2 / 3$ creates correlation between the dimensions


In [6]:
# Visualize data distribution
fig = Figure(size=(800, 400))
ax1 = Axis(fig[1, 1], title="x₁ Distribution", xlabel="x₁", ylabel="Density")
hist!(ax1, data[1, :], bins=100, normalization=:pdf, color=:steelblue)
ax2 = Axis(fig[1, 2], title="x₂ Distribution", xlabel="x₂", ylabel="Density")
hist!(ax2, data[2, :], bins=100, normalization=:pdf, color=:steelblue)
display(fig)


GLMakie.Screen(...)

In [ ]:
# Test score computation on a grid
test_point = [0.0, 0.0]
tmp = gaussian_mixture_score(test_point, data, sigma)
println("Gaussian mixture score at origin: ", tmp)
println("Exact score at origin: ", exact_score(test_point))


---

## Part 4: Neural Network Architecture

We use a simple feedforward neural network with:
- **Input**: 2D position $\mathbf{x}$
- **Hidden layer**: 50 neurons with Swish activation
- **Linear bypass**: A direct linear connection from input to output
- **Output**: 2D score estimate $\mathbf{s}_\theta(\mathbf{x})$

The **linear bypass** helps the network learn linear score components directly, which is important since the score has both linear and nonlinear terms:

$$\mathbf{s}(\mathbf{x}) = \begin{bmatrix} x_1 - x_1^3 - x_2/3 \\ x_2 - x_2^3 - x_1/3 \end{bmatrix} = \underbrace{\begin{bmatrix} x_1 - x_2/3 \\ x_2 - x_1/3 \end{bmatrix}}_{\text{linear}} + \underbrace{\begin{bmatrix} -x_1^3 \\ -x_2^3 \end{bmatrix}}_{\text{nonlinear}}$$


In [7]:
# ─────────────────────────────────────────────────────────────────────────────
# Define Network
# ─────────────────────────────────────────────────────────────────────────────

Nθ = 2        # Input dimension (2D position)
Nθᴴ = 50      # Hidden dimension
Nout = 2      # Output dimension (2D score)

# Create network with linear bypass for better learning
network = OneLayerNetworkWithLinearByPass(Nθ, Nout, Nθᴴ)
dnetwork = deepcopy(network)           # For storing gradients
smoothed_network = deepcopy(network)   # For exponential moving average

println("Network architecture: $(Nθ) → $(Nθᴴ) → $(Nout) with linear bypass")


Network architecture: 2 → 50 → 2 with linear bypass


In [8]:
# Test loss computation
test_data = data[:, 1:10]
test_noise = randn(2, 10)
initial_loss = loss_function(network, test_data, σ, test_noise)
println("Initial loss on test batch: $(round(initial_loss, digits=3))")

# Create convenience functions for score computation
gmscore(x) = gaussian_mixture_score(x, data, sigma)
exactscore(x) = exact_score(x)


Initial loss on test batch: 724.529


exactscore (generic function with 1 method)

---

## Part 5: Training the Network

We train using:
- **Adam optimizer**: Adaptive learning rates for each parameter
- **Mini-batch gradient descent**: Process data in batches of 20 samples
- **Enzyme.jl**: Automatic differentiation for computing gradients
- **Exponential moving average**: Smooth network parameters for stability

### Training Loop Details

For each epoch:
1. Shuffle training data into mini-batches
2. For each batch:
   - Sample fresh noise $\mathbf{z}$
   - Compute denoising loss
   - Backpropagate and update weights
3. Evaluate loss on held-out test data
4. Update exponential moving average of weights


In [9]:
# ─────────────────────────────────────────────────────────────────────────────
# Training
# ─────────────────────────────────────────────────────────────────────────────

adam = Adam(network)          # Initialize Adam optimizer
batchsize = 20                # Samples per batch
loss_list = Float64[]         # Track training loss
test_loss_list = Float64[]    # Track test loss
epochs = 200                  # Number of training epochs
network_parameters = copy(parameters(network))  # For EMA smoothing

println("Starting training for $(epochs) epochs...")
println("Batch size: $(batchsize), Total samples: $(M)")

for i in ProgressBar(1:epochs)
    # Split data: odd indices for training, even for testing
    shuffled_list = chunk_list(shuffle(1:2:M), batchsize)
    shuffled_test_list = chunk_list(shuffle(2:2:M), batchsize)
    loss_value = 0.0
    N = length(shuffled_list)
    
    # ─── Training Pass ───
    for permuted_list in shuffled_list
        θbatch = data[:, permuted_list]  # 2×batchsize matrix
        zero!(dnetwork)                   # Reset gradients
        zs = randn(2, length(permuted_list))  # Sample fresh noise
        
        # Compute gradients via Enzyme autodiff
        autodiff(Enzyme.Reverse, loss_function, Active, 
                 DuplicatedNoNeed(network, dnetwork), 
                 Const(θbatch), Const(sigma), Const(zs))
        
        # Update network with Adam
        update!(adam, network, dnetwork)
        loss_value += loss_function(network, θbatch, sigma, zs) / N
    end
    push!(loss_list, loss_value)
    
    # ─── Test Pass ───
    loss_value = 0.0
    N = length(shuffled_test_list)
    for permuted_list in shuffled_test_list
        θbatch = data[:, permuted_list]
        zs = randn(2, length(permuted_list))
        loss_value += loss_function(network, θbatch, sigma, zs) / N
    end
    push!(test_loss_list, loss_value)
    
    # ─── Exponential Moving Average ───
    m = 0.9  # Smoothing factor
    network_parameters .= m * network_parameters + (1 - m) * parameters(network)
    set_parameters!(smoothed_network, network_parameters)
end

println("\nTraining complete!")
println("Final training loss: $(round(loss_list[end], digits=4))")
println("Final test loss: $(round(test_loss_list[end], digits=4))")


Starting training for 200 epochs...
Batch size: 20, Total samples: 10000


0.0%┣                                             ┫ 0/200 [00:01<-3:-37, -1s/it]
0.5%┣▏                                         ┫ 1/200 [00:08<Inf:Inf, InfGs/it]
1.5%┣▊                                              ┫ 3/200 [00:08<13:56, 4s/it]
2.5%┣█▏                                             ┫ 5/200 [00:09<06:58, 2s/it]
3.5%┣█▋                                             ┫ 7/200 [00:09<04:38, 1s/it]
4.5%┣██▏                                            ┫ 9/200 [00:09<03:28, 1s/it]
5.5%┣██▌                                           ┫ 11/200 [00:09<02:46, 1it/s]
6.5%┣███                                           ┫ 13/200 [00:09<02:18, 1it/s]
7.5%┣███▌                                          ┫ 15/200 [00:09<01:58, 2it/s]
8.5%┣████                                          ┫ 17/200 [00:09<01:43, 2it/s]
9.5%┣████▍                                         ┫ 19/200 [00:09<01:31, 2it/s]
10.5%┣████▊                                        ┫ 21/200 [00:09<01:22, 2it/s]
11.5%┣█████▏                


Training complete!
Final training loss: 793.341
Final test loss: 811.2653


100.0%┣██████████████████████████████████████████┫ 200/200 [00:16<00:00, 13it/s]
100.0%┣██████████████████████████████████████████┫ 200/200 [00:16<00:00, 13it/s]


### Figure 2: Training Loss Curves

This figure shows the **learning dynamics** during training.

**What to observe:**
- **Log-scale y-axis**: Helps visualize orders of magnitude improvement
- **Training loss (blue)**: Should decrease steadily
- **Test loss (red)**: Should follow training loss closely
- **Convergence**: Both losses stabilize after sufficient epochs
- **No overfitting**: If test loss diverges from training loss, the model is overfitting

**Note**: The denoising loss has an irreducible minimum — even the perfect score function has nonzero loss due to the stochastic noise $\mathbf{z}$.


In [10]:
# ─────────────────────────────────────────────────────────────────────────────
# Plot Training Loss
# ─────────────────────────────────────────────────────────────────────────────

loss_fig = Figure(size=(700, 400))
ax = Axis(loss_fig[1, 1]; 
          title="Training Progress", 
          xlabel="Epoch", 
          ylabel="Log₁₀(Loss)")
scatter!(ax, log10.(loss_list); color=:steelblue, markersize=6, label="Training Loss")
scatter!(ax, log10.(test_loss_list); color=:crimson, markersize=6, label="Test Loss")
axislegend(ax, position=:rt)
display(loss_fig)


GLMakie.Screen(...)

---

## Part 6: Evaluating the Learned Score

We compare three score functions:
1. **Network**: Our trained neural network $\mathbf{s}_\theta$
2. **Gaussian Mixture**: The kernel density estimate score
3. **Exact**: The analytical score from $-\nabla V$

Lower loss indicates better score estimation.


In [11]:
# ─────────────────────────────────────────────────────────────────────────────
# Evaluate Final Losses
# ─────────────────────────────────────────────────────────────────────────────

"""
Evaluate denoising loss for any score function.
This allows us to compare different score estimators on the same footing.
"""
function denoising_loss_with_fn(score_fn, data, σ, noises)
    batchsize = size(data, 2)
    lossval = 0.0
    
    for i in 1:batchsize
        x = data[:, i]
        z = noises[:, i]
        x̃ = x .+ σ .* z
        ŷ = score_fn(x̃)
        lossval += sum((ŷ .+ z ./ σ).^2) / batchsize
    end
    
    return lossval
end

# Define score functions for evaluation
network_score_fn(x) = predict(network, x)

# Average over multiple noise realizations for stable estimates
skip = 50
number_of_samples = 10

println("Evaluating score functions (averaging over $(number_of_samples) noise samples)...")

l1 = mean([denoising_loss_with_fn(network_score_fn, data[:, 1:skip:end], σ, randn(2, M)) 
           for _ in 1:number_of_samples])
l2 = mean([denoising_loss_with_fn(gmscore, data[:, 1:skip:end], σ, randn(2, M)) 
           for _ in ProgressBar(1:number_of_samples)])
l3 = mean([denoising_loss_with_fn(exactscore, data[:, 1:skip:end], σ, randn(2, M)) 
           for _ in 1:number_of_samples])

println("\n" * "="^50)
println("SCORE FUNCTION COMPARISON")
println("="^50)
println("Network Loss:          $(round(l1, digits=2))")
println("Gaussian Mixture Loss: $(round(l2, digits=2))")
println("Exact Score Loss:      $(round(l3, digits=2))")
println("="^50)


Evaluating score functions (averaging over 10 noise samples)...


0.0%┣                                               ┫ 0/10 [00:00<00:00, -0s/it]
10.0%┣████▏                                     ┫ 1/10 [00:00<Inf:Inf, InfGs/it]
20.0%┣█████████▍                                     ┫ 2/10 [00:00<00:04, 2it/s]
30.0%┣██████████████                                 ┫ 3/10 [00:01<00:02, 3it/s]
40.0%┣██████████████████▉                            ┫ 4/10 [00:01<00:02, 4it/s]
50.0%┣███████████████████████▌                       ┫ 5/10 [00:01<00:01, 4it/s]
60.0%┣████████████████████████████▏                  ┫ 6/10 [00:01<00:01, 5it/s]
70.0%┣█████████████████████████████████              ┫ 7/10 [00:01<00:01, 5it/s]
80.0%┣█████████████████████████████████████▋         ┫ 8/10 [00:01<00:00, 5it/s]
90.0%┣██████████████████████████████████████████▎    ┫ 9/10 [00:02<00:00, 5it/s]



SCORE FUNCTION COMPARISON
Network Loss:          796.06
Gaussian Mixture Loss: 756.97
Exact Score Loss:      810.0


100.0%┣█████████████████████████████████████████████┫ 10/10 [00:02<00:00, 5it/s]
100.0%┣█████████████████████████████████████████████┫ 10/10 [00:02<00:00, 5it/s]


---

## Part 7: Visualizing the Learned Score Field

### Figure 3: Score Vector Fields

This figure compares the **score vector fields** from all three methods.

**How to read these plots:**
- **Arrows**: Point in the direction of increasing probability
- **Arrow direction**: The score direction $\mathbf{s}(\mathbf{x}) / \|\mathbf{s}(\mathbf{x})\|$
- **Gray dots**: Subsampled data points showing the true distribution

**What to observe:**
- All three methods should show arrows pointing **toward the modes** (high-density regions)
- Near the origin, arrows point outward (low probability region between modes)
- Near $(\pm 1, \pm 1)$, arrows point inward (high probability modes)
- The network should approximate the exact score pattern


In [12]:
# ─────────────────────────────────────────────────────────────────────────────
# Visualize Learned Score Field
# ─────────────────────────────────────────────────────────────────────────────

# Create grid for visualization
x_range = range(-2, 2, length=30)
y_range = range(-2, 2, length=30)

# Preallocate score arrays
network_scores_u = zeros(length(x_range), length(y_range))
network_scores_v = zeros(length(x_range), length(y_range))
gm_scores_u = zeros(length(x_range), length(y_range))
gm_scores_v = zeros(length(x_range), length(y_range))
exact_scores_u = zeros(length(x_range), length(y_range))
exact_scores_v = zeros(length(x_range), length(y_range))

println("Computing score fields on grid...")
for (i, x) in ProgressBar(enumerate(x_range))
    for (j, y) in enumerate(y_range)
        # Network score
        net_score = predict(network, [x, y])
        network_scores_u[i, j] = net_score[1]
        network_scores_v[i, j] = net_score[2]
        
        # Gaussian mixture score
        gm_score = gaussian_mixture_score([x, y], data, sigma)
        gm_scores_u[i, j] = gm_score[1]
        gm_scores_v[i, j] = gm_score[2]
        
        # Exact score
        exact_score_val = exact_score([x, y])
        exact_scores_u[i, j] = exact_score_val[1]
        exact_scores_v[i, j] = exact_score_val[2]
    end
end


Computing score fields on grid...


0.0%┣                                               ┫ 0/30 [00:00<00:00, -0s/it]
6.7%┣███▏                                           ┫ 2/30 [00:00<00:02, 16it/s]
13.3%┣██████▏                                       ┫ 4/30 [00:00<00:01, 26it/s]
20.0%┣█████████▏                                    ┫ 6/30 [00:00<00:01, 28it/s]
30.0%┣█████████████▉                                ┫ 9/30 [00:00<00:01, 32it/s]
40.0%┣██████████████████                           ┫ 12/30 [00:00<00:01, 35it/s]
46.7%┣█████████████████████                        ┫ 14/30 [00:00<00:00, 36it/s]
53.3%┣████████████████████████                     ┫ 16/30 [00:00<00:00, 36it/s]
63.3%┣████████████████████████████▌                ┫ 19/30 [00:00<00:00, 38it/s]
73.3%┣█████████████████████████████████            ┫ 22/30 [00:01<00:00, 38it/s]
80.0%┣████████████████████████████████████         ┫ 24/30 [00:01<00:00, 38it/s]
86.7%┣███████████████████████████████████████      ┫ 26/30 [00:01<00:00, 38it/s]
96.7%┣██████████████████████

In [13]:
# Normalize vectors for uniform arrow lengths (shows direction only)
function normalize_vectors(u, v, scale=0.15)
    mag = sqrt.(u.^2 .+ v.^2)
    mag[mag .< 1e-10] .= 1e-10  # Avoid division by zero
    return u .* scale ./ mag, v .* scale ./ mag
end

net_u_scaled, net_v_scaled = normalize_vectors(network_scores_u, network_scores_v)
gm_u_scaled, gm_v_scaled = normalize_vectors(gm_scores_u, gm_scores_v)
exact_u_scaled, exact_v_scaled = normalize_vectors(exact_scores_u, exact_scores_v)

# Create grid coordinate matrices
x_grid = [x for x in x_range, y in y_range]
y_grid = [y for x in x_range, y in y_range]


30×30 Matrix{Float64}:
 -2.0  -1.86207  -1.72414  -1.58621  …  1.58621  1.72414  1.86207  2.0
 -2.0  -1.86207  -1.72414  -1.58621     1.58621  1.72414  1.86207  2.0
 -2.0  -1.86207  -1.72414  -1.58621     1.58621  1.72414  1.86207  2.0
 -2.0  -1.86207  -1.72414  -1.58621     1.58621  1.72414  1.86207  2.0
 -2.0  -1.86207  -1.72414  -1.58621     1.58621  1.72414  1.86207  2.0
 -2.0  -1.86207  -1.72414  -1.58621  …  1.58621  1.72414  1.86207  2.0
 -2.0  -1.86207  -1.72414  -1.58621     1.58621  1.72414  1.86207  2.0
 -2.0  -1.86207  -1.72414  -1.58621     1.58621  1.72414  1.86207  2.0
 -2.0  -1.86207  -1.72414  -1.58621     1.58621  1.72414  1.86207  2.0
 -2.0  -1.86207  -1.72414  -1.58621     1.58621  1.72414  1.86207  2.0
 -2.0  -1.86207  -1.72414  -1.58621  …  1.58621  1.72414  1.86207  2.0
 -2.0  -1.86207  -1.72414  -1.58621     1.58621  1.72414  1.86207  2.0
 -2.0  -1.86207  -1.72414  -1.58621     1.58621  1.72414  1.86207  2.0
  ⋮                                  ⋱                

In [14]:
# Plot score vector field comparison
fig = Figure(size=(1800, 600))

# Network score field
ax1 = Axis(fig[1, 1], 
           title="Network Score Field\n(Loss: $(round(l1, digits=1)))", 
           xlabel="x₁", ylabel="x₂", 
           aspect=DataAspect(), limits=(-2, 2, -2, 2))
scatter!(ax1, data[1, 1:100:end], data[2, 1:100:end], 
         markersize=3, alpha=0.3, color=:gray)
arrows!(ax1, vec(x_grid), vec(y_grid), vec(net_u_scaled), vec(net_v_scaled), 
        color=:steelblue, linewidth=1.5, arrowsize=8)

# Gaussian mixture score field
ax2 = Axis(fig[1, 2], 
           title="Gaussian Mixture Score Field\n(Loss: $(round(l2, digits=1)))", 
           xlabel="x₁", ylabel="x₂", 
           aspect=DataAspect(), limits=(-2, 2, -2, 2))
scatter!(ax2, data[1, 1:100:end], data[2, 1:100:end], 
         markersize=3, alpha=0.3, color=:gray)
arrows!(ax2, vec(x_grid), vec(y_grid), vec(gm_u_scaled), vec(gm_v_scaled), 
        color=:crimson, linewidth=1.5, arrowsize=8)

# Exact score field
ax3 = Axis(fig[1, 3], 
           title="Exact Score Field\n(Loss: $(round(l3, digits=1)))", 
           xlabel="x₁", ylabel="x₂", 
           aspect=DataAspect(), limits=(-2, 2, -2, 2))
scatter!(ax3, data[1, 1:100:end], data[2, 1:100:end], 
         markersize=3, alpha=0.3, color=:gray)
arrows!(ax3, vec(x_grid), vec(y_grid), vec(exact_u_scaled), vec(exact_v_scaled), 
        color=:forestgreen, linewidth=1.5, arrowsize=8)

display(fig)


┌ Warning: `arrows` are deprecated in favor of `arrows2d` and `arrows3d`.
└ @ Makie ~/.julia/packages/Makie/Vn16E/src/basic_recipes/arrows.jl:166
┌ Warning: arrowsize has been deprecated in favor of tipwidth and tiplength.
└ @ Makie ~/.julia/packages/Makie/Vn16E/src/basic_recipes/arrows.jl:206
┌ Warning: `arrows` are deprecated in favor of `arrows2d` and `arrows3d`.
└ @ Makie ~/.julia/packages/Makie/Vn16E/src/basic_recipes/arrows.jl:166
┌ Warning: arrowsize has been deprecated in favor of tipwidth and tiplength.
└ @ Makie ~/.julia/packages/Makie/Vn16E/src/basic_recipes/arrows.jl:206
┌ Warning: `arrows` are deprecated in favor of `arrows2d` and `arrows3d`.
└ @ Makie ~/.julia/packages/Makie/Vn16E/src/basic_recipes/arrows.jl:166
┌ Warning: arrowsize has been deprecated in favor of tipwidth and tiplength.
└ @ Makie ~/.julia/packages/Makie/Vn16E/src/basic_recipes/arrows.jl:206


GLMakie.Screen(...)

### Figure 4: Score Magnitude Heatmaps

This figure shows the **magnitude** of the score at each point: $\|\mathbf{s}(\mathbf{x})\|$.

**What to observe:**
- **Low magnitude** (dark) near modes: At high probability regions, the score is small
- **High magnitude** (bright) away from modes: The score pushes samples toward high-density regions
- **Saddle point at origin**: The score has a critical point at $(0, 0)$
- **Symmetry**: The pattern should be symmetric due to the symmetric potential


In [15]:
# ─────────────────────────────────────────────────────────────────────────────
# Plot Score Magnitude Comparison
# ─────────────────────────────────────────────────────────────────────────────

# Compute score magnitudes
network_mag = sqrt.(network_scores_u.^2 .+ network_scores_v.^2)
gm_mag = sqrt.(gm_scores_u.^2 .+ gm_scores_v.^2)
exact_mag = sqrt.(exact_scores_u.^2 .+ exact_scores_v.^2)

# Common color scale based on exact score
cmin, cmax = quantile(vec(exact_mag), [0.01, 0.99])

fig_mag = Figure(size=(1800, 500))

ax1 = Axis(fig_mag[1, 1], title="Network Score Magnitude", 
           xlabel="x₁", ylabel="x₂", 
           aspect=DataAspect(), limits=(-2, 2, -2, 2))
hm1 = heatmap!(ax1, x_range, y_range, network_mag, 
               colormap=:viridis, colorrange=(0, cmax))

ax2 = Axis(fig_mag[1, 2], title="Gaussian Mixture Score Magnitude", 
           xlabel="x₁", ylabel="x₂", 
           aspect=DataAspect(), limits=(-2, 2, -2, 2))
hm2 = heatmap!(ax2, x_range, y_range, gm_mag, 
               colormap=:viridis, colorrange=(0, cmax))

ax3 = Axis(fig_mag[1, 3], title="Exact Score Magnitude", 
           xlabel="x₁", ylabel="x₂", 
           aspect=DataAspect(), limits=(-2, 2, -2, 2))
hm3 = heatmap!(ax3, x_range, y_range, exact_mag, 
               colormap=:viridis, colorrange=(0, cmax))

Colorbar(fig_mag[1, 4], hm3, label="Score Magnitude ||s(x)||")

display(fig_mag)


GLMakie.Screen(...)

---

## Part 8: Sampling with Langevin Dynamics

Now we use our learned score to **generate new samples** from the distribution!

### Langevin Dynamics

Given the score function $\mathbf{s}(\mathbf{x})$, we can sample from $p(\mathbf{x})$ using the **overdamped Langevin SDE**:

$$d\mathbf{x}_t = \mathbf{s}(\mathbf{x}_t) dt + \sqrt{2} \, d\mathbf{W}_t$$

Discretized with time step $\Delta t$:

$$\mathbf{x}_{t+1} = \mathbf{x}_t + \Delta t \cdot \mathbf{s}(\mathbf{x}_t) + \sqrt{2 \Delta t} \cdot \boldsymbol{\xi}_t$$

where $\boldsymbol{\xi}_t \sim \mathcal{N}(0, I)$.

### Runge-Kutta Integration

We use **4th-order Runge-Kutta (RK4)** for the deterministic part to improve accuracy:

$$\mathbf{k}_1 = \mathbf{s}(\mathbf{x}_t)$$
$$\mathbf{k}_2 = \mathbf{s}(\mathbf{x}_t + \frac{\Delta t}{2} \mathbf{k}_1)$$
$$\mathbf{k}_3 = \mathbf{s}(\mathbf{x}_t + \frac{\Delta t}{2} \mathbf{k}_2)$$
$$\mathbf{k}_4 = \mathbf{s}(\mathbf{x}_t + \Delta t \cdot \mathbf{k}_3)$$
$$\mathbf{x}_{t+1} = \mathbf{x}_t + \frac{\Delta t}{6}(\mathbf{k}_1 + 2\mathbf{k}_2 + 2\mathbf{k}_3 + \mathbf{k}_4) + \sqrt{2 \Delta t} \cdot \boldsymbol{\xi}_t$$


In [16]:
# ─────────────────────────────────────────────────────────────────────────────
# Sampling from the Learned Score Field
# ─────────────────────────────────────────────────────────────────────────────

# Use network as our score function
∇V(x) = predict(network, x)

# Langevin dynamics parameters
ϵ = sqrt(2)    # Noise coefficient (√2 for standard Langevin)
Nₜ = 1000      # Number of time steps
Nₑ = 1000      # Number of ensemble members (parallel chains)
Δt = 0.1       # Time step size

# Initialize from standard Gaussian
x₀ = randn(2, Nₑ)

println("Running Langevin dynamics...")
println("  Samples: $(Nₑ), Steps: $(Nₜ), Δt: $(Δt)")

for t in ProgressBar(1:Nₜ)
    𝒩 = randn(2, Nₑ)  # Fresh noise for all samples
    
    for ω in 1:Nₑ
        x = x₀[:, ω]
        
        # Runge-Kutta 4 stages for deterministic drift
        k1 = ∇V(x)
        k2 = ∇V(x + Δt / 2 * k1)
        k3 = ∇V(x + Δt / 2 * k2)
        k4 = ∇V(x + Δt * k3)
        
        # RK4 update (deterministic part)
        x₀[:, ω] .= x + Δt / 6 * (k1 + 2 * k2 + 2 * k3 + k4)
        
        # Add stochastic noise (Brownian motion)
        x₀[:, ω] .+= ϵ * √Δt * 𝒩[:, ω]
    end
end

println("\nSampling complete!")


Running Langevin dynamics...
  Samples: 1000, Steps: 1000, Δt: 0.1


0.0%┣                                             ┫ 0/1.0k [00:00<00:00, -0s/it]
0.1%┣                                         ┫ 1/1.0k [00:00<Inf:Inf, InfGs/it]
1.0%┣▍                                           ┫ 10/1.0k [00:00<00:20, 50it/s]
1.5%┣▋                                           ┫ 15/1.0k [00:00<00:16, 61it/s]
2.4%┣█                                           ┫ 24/1.0k [00:00<00:12, 81it/s]
3.3%┣█▌                                          ┫ 33/1.0k [00:00<00:10, 94it/s]
3.9%┣█▊                                          ┫ 39/1.0k [00:00<00:10, 96it/s]
4.8%┣██                                         ┫ 48/1.0k [00:00<00:09, 104it/s]
5.4%┣██▎                                        ┫ 54/1.0k [00:01<00:09, 106it/s]
6.3%┣██▊                                        ┫ 63/1.0k [00:01<00:08, 112it/s]
7.1%┣███                                        ┫ 71/1.0k [00:01<00:08, 116it/s]
7.8%┣███▍                                       ┫ 78/1.0k [00:01<00:08, 117it/s]
8.6%┣███▊                   


Sampling complete!


99.9%┣█████████████████████████████████████████┫ 999/1.0k [00:07<00:00, 146it/s]
100.0%┣███████████████████████████████████████┫ 1.0k/1.0k [00:07<00:00, 146it/s]
100.0%┣███████████████████████████████████████┫ 1.0k/1.0k [00:07<00:00, 146it/s]


### Figure 5: Generated Samples vs Original Data

This figure compares the **marginal distributions** of our generated samples (blue) with the original training data (red).

**What to observe:**
- **Mode locations**: Generated samples should cluster around $x \approx \pm 1$
- **Mode weights**: The relative heights of the two peaks should match
- **Shape**: The overall distribution shape should be similar
- **Quality indicator**: Good overlap means the network learned the distribution well

**Common issues:**
- Missing modes: Network didn't capture all peaks
- Mode collapse: Samples cluster at single peak
- Spread mismatch: Wrong variance indicates poor score magnitude learning


In [17]:
# ─────────────────────────────────────────────────────────────────────────────
# Compare Generated Samples with Original Data
# ─────────────────────────────────────────────────────────────────────────────

binsize = 30

fig_samples = Figure(size=(900, 400))

ax1 = Axis(fig_samples[1, 1], title="x₁ Marginal Distribution", 
           xlabel="x₁", ylabel="Density")
hist!(ax1, x₀[1, :], bins=binsize, normalization=:pdf, 
      color=(:steelblue, 0.6), label="Generated")
hist!(ax1, data[1, :], bins=binsize, normalization=:pdf, 
      color=(:crimson, 0.4), label="Original")
axislegend(ax1, position=:rt)

ax2 = Axis(fig_samples[1, 2], title="x₂ Marginal Distribution", 
           xlabel="x₂", ylabel="Density")
hist!(ax2, x₀[2, :], bins=binsize, normalization=:pdf, 
      color=(:steelblue, 0.6), label="Generated")
hist!(ax2, data[2, :], bins=binsize, normalization=:pdf, 
      color=(:crimson, 0.4), label="Original")
axislegend(ax2, position=:rt)

display(fig_samples)


GLMakie.Screen(...)

In [18]:
# ─────────────────────────────────────────────────────────────────────────────
# Statistical Comparison
# ─────────────────────────────────────────────────────────────────────────────

println("="^60)
println("STATISTICAL COMPARISON: Generated vs Original")
println("="^60)
println()
println("Mean:")
println("  Generated: ", round.(vec(mean(x₀, dims=2)), digits=4))
println("  Original:  ", round.(vec(mean(data, dims=2)), digits=4))
println()
println("Covariance (Generated):")
cov_gen = cov(x₀, dims=2)
println("  [", round(cov_gen[1,1], digits=4), "  ", round(cov_gen[1,2], digits=4), "]")
println("  [", round(cov_gen[2,1], digits=4), "  ", round(cov_gen[2,2], digits=4), "]")
println()
println("Covariance (Original):")
cov_orig = cov(data, dims=2)
println("  [", round(cov_orig[1,1], digits=4), "  ", round(cov_orig[1,2], digits=4), "]")
println("  [", round(cov_orig[2,1], digits=4), "  ", round(cov_orig[2,2], digits=4), "]")
println("="^60)


STATISTICAL COMPARISON: Generated vs Original

Mean:
  Generated: [-0.146, 0.1305]
  Original:  [-0.0119, 0.0008]

Covariance (Generated):
  [1.2442  -0.4787]
  [-0.4787  1.3027]

Covariance (Original):
  [1.2342  -0.3766]
  [-0.3766  1.2231]


---

## Summary

### What We Learned

1. **Score functions** $\mathbf{s}(\mathbf{x}) = \nabla \log p(\mathbf{x})$ encode the gradient of log-probability

2. **Denoising Score Matching** provides a practical training objective:
   $$\mathcal{L}_{\text{DSM}} = \mathbb{E}\left[ \left\| \mathbf{s}_\theta(\mathbf{x} + \sigma\mathbf{z}) + \frac{\mathbf{z}}{\sigma} \right\|^2 \right]$$

3. **Neural networks** can approximate score functions effectively

4. **Langevin dynamics** enables sampling using only the score function

### Extensions

This tutorial covers the basics. Modern score-based models extend these ideas:

- **Multiple noise levels**: Train on a range of $\sigma$ values (Song & Ermon, 2019)
- **Continuous diffusion**: Use SDEs with time-varying noise (Song et al., 2021)
- **Conditional generation**: Add conditioning variables for controlled generation
- **Latent diffusion**: Work in compressed latent spaces (Rombach et al., 2022)

### References

- Vincent, P. (2011). "A Connection Between Score Matching and Denoising Autoencoders"
- Song, Y., & Ermon, S. (2019). "Generative Modeling by Estimating Gradients of the Data Distribution"
- Song, Y., et al. (2021). "Score-Based Generative Modeling through Stochastic Differential Equations"
